In [1]:
import pandas as pd
data = pd.read_csv("ny_state_school3.csv")
data = data[['School','Principal_Investigator_1_First_Name','Principal_Investigator_1_Last_Name','Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name','Total_Amount_of_Payment_USDollars']]
data.columns = ['School','PI_first','PI_last','Company','Money']

In [2]:
data['PI'] = data['PI_first'] + ' ' + data['PI_last']
data1 = data.groupby(['PI'])['Money'].sum().reset_index()
data1.columns = ['PI','PI_money']
data2 = data.groupby(['School'])['Money'].sum().reset_index()
data2.columns = ['School','School_money']
data3 = data.groupby(['Company'])['Money'].sum().reset_index()
data3.columns = ['Company','Company_money']

In [3]:
data = data.drop(['Money'], axis = 1).merge(data1, how = 'left', on = 'PI')
data = data.merge(data2, how = 'left', on = 'School')
data = data.merge(data3, how = 'left', on = 'Company')

In [4]:
data = data.dropna()
data = data.drop_duplicates()
data = data[data['School']!='others']
data = data.drop(['PI_first','PI_last'], axis = 1)
data['PI'] = data['PI'].map(lambda x: " ".join([i[0].upper()+i[1:].lower() for i in x.split(" ") if type(i) is str]))


In [5]:
data['PI_score'] = data['PI_money']/max(data['PI_money'])
data['School_score'] = data['School_money']/max(data['School_money'])
data['Company_score'] = data['Company_money']/max(data['Company_money'])

In [6]:
data['sort'] = data['PI_score'] + data['School_score'] + data['Company_score']
data.sort_values(by = ['sort'], axis = 0,ascending = False)
data = data.iloc[:500]

In [7]:
#construct dictionary
dic = list(data.School.unique())+list(data.PI.unique())
dics = [{"name":b,"ID":a,"PI":b in data.PI.unique()} for a, b in enumerate(dic)]

In [8]:
nodes = []
for i in dics:
    node = {}
    if i['PI']:
        node['money'] = int(data[data.PI == i['name']]['PI_money'].iloc[-1]/1000)
        node['score'] = data[data.PI == i['name']]['PI_score'].iloc[-1]
        node['size'] = node['score']*60
        node['type'] = "circle"
    else:
        node['money'] = int(data[data.School == i['name']]['School_money'].iloc[-1]/1000)
        node['score'] = data[data.School == i['name']]['School_score'].iloc[-1]
        node['size'] = node['score']*60
        node['type'] = "square"
    node['id'] = i['name']
    nodes = nodes + [node]

In [9]:
dic = {a['name']:a['ID'] for a in dics}

In [10]:
links = []
for i in range(data.shape[0]):
    links = links + [{"source": dic[data.School.iloc[i]], "target": dic[data.PI.iloc[i]]}]
outputs = {"graph":[], "links":links, "nodes":nodes}


In [11]:
import json
print(json.dumps(outputs,indent=4))

{
    "graph": [],
    "nodes": [
        {
            "type": "square",
            "score": 0.12786002939197036,
            "size": 7.671601763518222,
            "id": "university of rochester medical center",
            "money": 5669
        },
        {
            "type": "square",
            "score": 0.24020744121964202,
            "size": 14.412446473178521,
            "id": "sinai",
            "money": 10651
        },
        {
            "type": "square",
            "score": 0.40839279664418726,
            "size": 24.503567798651236,
            "id": "cornell",
            "money": 18108
        },
        {
            "type": "square",
            "score": 0.0414258407207803,
            "size": 2.4855504432468183,
            "id": "albert einstein college of medicine",
            "money": 1836
        },
        {
            "type": "square",
            "score": 0.050459345745546426,
            "size": 3.0275607447327855,
            "id": "suny",
        